In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('./data/sample_submission.csv')

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df.columns.values.tolist()

In [ ]:
columns = df.columns.values.tolist()

In [ ]:
columns.pop(0)

In [ ]:
from data import *

In [ ]:
train_data = TrainDataset('./data/train', './data/labels.csv')

In [ ]:
train_data.class_to_idx

In [ ]:
list(train_data.class_to_idx.keys())

In [ ]:
df.columns.values.tolist() == list(train_data.class_to_idx.keys())

In [ ]:
columns == list(train_data.class_to_idx.keys())

In [ ]:
from model import *

In [ ]:
model = ResNet("resnet18", pretrained=True)
data = DogData()

In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger(".", default_hp_metric=False)

In [ ]:
import pytorch_lightning as pl

trainer = pl.Trainer(max_epochs=1, logger=logger)

In [ ]:
import torch

torch.set_float32_matmul_precision('medium')

In [ ]:
trainer.fit(model, datamodule=data)

In [ ]:
trainer.test(model, datamodule=data)

In [ ]:
predictions = trainer.predict(model, data)

In [ ]:
predict_data = PredictDataset('./data/test', './data/sample_submission.csv')

In [ ]:
predict_data[5]

In [ ]:
len(predictions)

In [ ]:
pred = predictions[0][0]

In [ ]:
pred

In [ ]:
torch.tensor(pred).argmax()

In [ ]:
pred[52]

In [ ]:
torch.tensor(pred).argmin()

In [ ]:
pred[58]

In [ ]:
sum(pred)

In [ ]:
s = sum([i for i in pred if i > 0])

In [ ]:
res = [i / s if i > 0 else 0 for i in pred]

In [ ]:
[i for i in res if i > 0]

In [ ]:
[i for i in pred if i > 1]

In [ ]:
for row in torch.tensor([[1,2],[3,4]]):
    print(row)

In [ ]:
[i.argmax().item() for row in predictions for i in row]

In [ ]:
predictions[0]

In [ ]:
value, position = predictions[0].max(dim=1, keepdim=True)

In [ ]:
torch.where(predictions[0] == value, 1.0, 0.0)

In [ ]:
def max_to_one(tensor):
    # 求每一行的最大值
    row_max_values, _ = torch.max(tensor, dim=1, keepdim=True)
    
    # 将每一行的最大值设为1，其它值设为0
    result_tensor = torch.where(tensor == row_max_values, torch.tensor(1.0), torch.tensor(0.0))
    
    return result_tensor

In [ ]:
max_to_one(predictions[0])

In [ ]:
[max_to_one(i) for i in predictions]

In [ ]:
import pandas as pd

df = pd.read_csv('./data/sample_submission.csv')

In [ ]:
imgs = df["id"]

In [ ]:
columns = df.columns.values.tolist()
columns.pop(0)

In [ ]:
len(columns)

In [ ]:
predictions_converted = [max_to_one(i) for i in predictions]

In [ ]:
predictions_write = [row.tolist() for item in predictions_converted for row in item]

In [ ]:
len(predictions_write)

In [ ]:
pd.Series(predictions_write, name=tuple(columns))

In [ ]:
df.columns.to_numpy()

In [ ]:
pd.Series(predictions_write, name=tuple(df.columns.to_numpy()))

In [ ]:
pd.DataFrame(predictions_write, columns=columns)

In [ ]:
imgs

In [ ]:
pd.concat([imgs, pd.DataFrame(predictions_write, columns=columns)], axis=1)

In [ ]:
pd.DataFrame([row for item in predictions for row in max_to_one(item)], columns=columns)

In [ ]:
pd.DataFrame([row.tolist() for item in predictions for row in max_to_one(item)], columns=columns)

In [ ]:
pd.DataFrame([row.tolist() for item in predictions for row in max_to_one(item)], columns=train_data.class_to_idx.keys())